In [74]:
from IPython.display import display, HTML, Image
from pathlib import Path
import requests
import json
import os

pp = lambda x: print(json.dumps(x, indent=2))

SKEMA_EQ2MML_SERVICE = os.environ.get("SKEMA_EQMML_ADDRESS", "http://eq2mml:8001")
SKEMA_RS_SERVICE = os.environ.get("SKEMA_RS_ADDRESS", "http://skema-rs:8080")

In [75]:
!pip install beautifulsoup4


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [76]:
from bs4 import BeautifulSoup, Comment


def clean_mml(mml: str) -> str:
    """Cleans/sterilizes pMML for AMR generation service"""
    soup = BeautifulSoup(mml, "html.parser")
    for comment in soup(text=lambda text: isinstance(text, Comment)):
    #for comment in soup.find(text=lambda t: isinstance(t, Comment)):
        comment.extract()
    return str(soup).replace("\n","")

## LaTeX equations $\rightarrow$ AMR


First, we'll convert our LaTeX equations to pMML:

In [88]:
# generate pMML
# NOTE: in Python, a literal backslash is represented as \\
tex_equations = ["\\frac{d x}{d t} = \\alpha x - \\beta x y", "\\frac{d y}{d t}= \\delta x y - \\gamma y"]
mml = [requests.post(f"{SKEMA_EQ2MML_SERVICE}/latex/mml", json={"tex_src": eq}).text for eq in tex_equations]
for eq in mml:
    print(eq)
    display(HTML(eq))

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block" alttext="\frac{d x}{d t} = \alpha x - \beta x y">
  <mfrac>
    <mrow>
      <mi>d</mi>
      <mi>x</mi>
    </mrow>
    <mrow>
      <mi>d</mi>
      <mi>t</mi>
    </mrow>
  </mfrac>
  <mo>=</mo>
  <mi>&#x03B1;<!-- α --></mi>
  <mi>x</mi>
  <mo>&#x2212;<!-- − --></mo>
  <mi>&#x03B2;<!-- β --></mi>
  <mi>x</mi>
  <mi>y</mi>
</math>


<math xmlns="http://www.w3.org/1998/Math/MathML" display="block" alttext="\frac{d y}{d t}= \delta x y - \gamma y">
  <mfrac>
    <mrow>
      <mi>d</mi>
      <mi>y</mi>
    </mrow>
    <mrow>
      <mi>d</mi>
      <mi>t</mi>
    </mrow>
  </mfrac>
  <mo>=</mo>
  <mi>&#x03B4;<!-- δ --></mi>
  <mi>x</mi>
  <mi>y</mi>
  <mo>&#x2212;<!-- − --></mo>
  <mi>&#x03B3;<!-- γ --></mi>
  <mi>y</mi>
</math>


In [ ]:
tex_equations

Next, we'll convert our pMML to AMR:

In [60]:
payload = {
    "mathml" : [clean_mml(eq) for eq in mml],
    "model" : "regnet"
}
r = requests.put(f"{SKEMA_RS_SERVICE}/models/PN", json=payload)
print(r)
print([clean_mml(eq) for eq in mml])
#requests.delete(f"{BASE_URL}/models/{MODEL_ID}").text

<Response [400]>
['<math alttext="\\frac{d x}{d t}= \\alpha x - \\beta x y" display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mfrac><mrow><mi>d</mi><mi>x</mi></mrow><mrow><mi>d</mi><mi>t</mi></mrow></mfrac><mo>=</mo><mi>α</mi><mi>x</mi><mo>−</mo><mi>β</mi><mi>x</mi><mi>y</mi></math>', '<math alttext="\\frac{d y}{d t}= \\delta x y - \\gamma y" display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mfrac><mrow><mi>d</mi><mi>y</mi></mrow><mrow><mi>d</mi><mi>t</mi></mrow></mfrac><mo>=</mo><mi>δ</mi><mi>x</mi><mi>y</mi><mo>−</mo><mi>γ</mi><mi>y</mi></math>']


## equations (PNG images) $\rightarrow$ AMR

First, we'll convert our equation images to pMML:

In [77]:
eq_1_path = Path("/data") / "skema" / "images" / "lotka-volterra-eq-1.png"
eq_2_path = Path("/data") / "skema" / "images" / "lotka-volterra-eq-2.png"

display(Image(filename=eq_1_path))

display(Image(filename=eq_2_path))

In [81]:
mml = []
for fp in [eq_1_path, eq_2_path]:
    files = {
      "data": open(eq_1_path, "rb"),
    }
    r = requests.post(f"{SKEMA_EQ2MML_SERVICE}/image/mml", files=files)
    print(r.text)
    mml.append(r.text)

<math> <mfrac> <mrow> <mi> d </mi> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03D1 </mi> </mrow> </mrow> <mrow> <mi> d </mi> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03D1 </mi> </mrow> </mrow> </mfrac> <mo> = </mo> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03B1 </mi> </mrow> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03D1 </mi> </mrow> <mo> &#x2212 </mo> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03B2 </mi> </mrow> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03D1 </mi> </mrow> <mo> = </mo> </math>
<math> <mfrac> <mrow> <mi> d </mi> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03D1 </mi> </mrow> </mrow> <mrow> <mi> d </mi> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03D1 </mi> </mrow> </mrow> </mfrac> <mo> = </mo> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03B1 </mi> </mrow> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03D1 </mi> </mrow> <mo> &#x2212 </mo> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03B2 </mi> </mrow> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03D1 </mi> </mrow> <mo> = </mo> </math>


In [84]:
payload = {
    "mathml" : mml,
    "model" : "regnet"
}
print(payload)
r = requests.put(f"{SKEMA_RS_SERVICE}/models/PN", json=payload)
print(r)

{'mathml': ['<math> <mfrac> <mrow> <mi> d </mi> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03D1 </mi> </mrow> </mrow> <mrow> <mi> d </mi> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03D1 </mi> </mrow> </mrow> </mfrac> <mo> = </mo> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03B1 </mi> </mrow> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03D1 </mi> </mrow> <mo> &#x2212 </mo> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03B2 </mi> </mrow> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03D1 </mi> </mrow> <mo> = </mo> </math>', '<math> <mfrac> <mrow> <mi> d </mi> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03D1 </mi> </mrow> </mrow> <mrow> <mi> d </mi> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03D1 </mi> </mrow> </mrow> </mfrac> <mo> = </mo> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03B1 </mi> </mrow> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03D1 </mi> </mrow> <mo> &#x2212 </mo> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03B2 </mi> </mrow> <mi> a </mi> <mi> s </mi> <mrow> <mi> &#x03D1 </mi> </mrow> <mo> = </mo> </math